# **Web Server Assigment**

# 1




## Overview
In this assignment, you will explore the fundamentals of socket programming for TCP connections in Python. You will learn how to create a socket, bind it to a specific address and port, and send and receive HTTP packets. Additionally, you will gain an understanding of the basic structure of HTTP header formats.

Your task is to develop a web server that processes one HTTP request at a time. The web server should:

1. Accept and parse the incoming HTTP request.
2. Retrieve the requested file from the server’s file system.
3. Construct an HTTP response message comprising the requested file preceded by appropriate header lines.
4. Send the response directly to the client.

If the requested file is not found on the server, the web server must return an HTTP "404 Not Found" message to the client.

## Code
Below is a skeleton code for the web server. Your task is to complete the code. The sections where you need to add your implementation are marked with `#-----#`. Some sections may require multiple lines of code.

## Running the Server
1. Place an HTML file (e.g., `HelloWorld.html`) in the same directory as the server program.
2. Execute the server program.
3. Determine the IP address of the machine running the server (e.g., `128.238.251.26`).
4. From a different machine or the same machine, open a web browser and navigate to the URL corresponding to the server. For example:
   ```
   http://128.238.251.26:6789/HelloWorld.html
   ```
   In this example, `HelloWorld.html` is the name of the HTML file you placed in the server directory. Note the port number `6789` used after the colon. Replace this port number with the one configured in your server code.

5. The browser should display the contents of `HelloWorld.html`. If the port number is omitted (e.g., `http://128.238.251.26/HelloWorld.html`), the browser will default to port 80. In this case, ensure your server is listening on port 80.
6. Test requesting a file that is not present on the server. You should receive a "404 Not Found" message in the browser.


## What to Submit
Submit the following items:

1. The complete server code.
2. Screenshots of your client browser demonstrating:
   - Successful retrieval of the HTML file content from the server.
   - Receiving a "404 Not Found" message for a non-existent file.

---

In [1]:
#import socket module
from socket import *
import sys # In order to terminate the program

serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(1)
#Prepare a sever socket
#_________#
print ('Ready to serve...')
while True:
	#Establish the connection
	print ('Ready to serve...')
	connectionSocket, addr = serverSocket.accept()

	try:
		message = connectionSocket.recv(1024).decode()
		print(message)
		filename = message.split()[1]
		f = open(filename[1:])
		outputdata = f.read()
		#Send one HTTP header line into socket
		connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())
		#Send the content of the requested file to the client
		for i in range(0, len(outputdata)):
			connectionSocket.send(outputdata[i].encode())
		connectionSocket.send("\r\n".encode())

		connectionSocket.close()
	except IOError:
		#Send response message for file not found
		connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
		connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>\r\n".encode())
		#Close client socket
	finally:
		connectionSocket.close()
		print('Connection closed')
serverSocket.close()
sys.exit() #Terminate the program after sending the corresponding data

# 2

Currently, the web server handles only one HTTP request at a time. Implement a multithreaded server
that is capable of serving multiple requests simultaneously. Using threading, first create a main thread
in which your modified server listens for clients at a fixed port. When it receives a TCP connection
request from a client, it will set up the TCP connection through another port and services the client
request in a separate thread. There will be a separate TCP connection in a separate thread for each
request/response pair.

In [2]:
from socket import *
import sys 
from threading import Thread
def client(connectionSocket, addr):
    try:
        message = connectionSocket.recv(1024).decode()
        print(message)
        if not message or len(message.split()) < 2:
            print("Empty or malformed HTTP request received.")
            connectionSocket.close()
            return
    
        filename = message.split()[1]
        f = open(filename[1:])
        outputdata = f.read()
        #Send one HTTP header line into socket
        connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())
        #Send the content of the requested file to the client
        for i in range(0, len(outputdata)):
            connectionSocket.send(outputdata[i].encode())
        connectionSocket.send("\r\n".encode())
        connectionSocket.close()

    except IOError:
		#Send response message for file not found
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
        connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>\r\n".encode())
		#Close client socket
    finally:
       connectionSocket.close()
       print('Connection closed')
    
def start_server(serverPort = 12000):
    serverSocket = socket(AF_INET, SOCK_STREAM)
    serverSocket.bind(("", serverPort))
    serverSocket.listen(1)
    print(f"Multi-threaded server is running on port {serverPort}...")
    try :
        while True:
            print ('Ready to serve...')
            connectionSocket, addr = serverSocket.accept()
            client_thread = Thread(target=client, args=(connectionSocket, addr))
            client_thread.start()
    except KeyboardInterrupt:
        print("\nServer shutting down...")
    finally:
        serverSocket.close()
        sys.exit()
if __name__ == "__main__":
    start_server()

# 3

The Multi Thread Web Server is now set and working but cannot handle changes in the source files. How can we look for changes in the source directory and access it in your code ? Explore Libraries in python

In [ ]:
#we can use watchdog library.#
!pip install watchdog


After you've accessed it in your code, Post a notification on the hosted http page to reload if any source directory changes

In [1]:
from socket import *
import sys
from threading import Thread
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os

# File Change Handler to detect file modifications
class FileChangeHandler(FileSystemEventHandler):
    def __init__(self, directory, server_script_name):
        self.directory = directory
        self.server_script_name = server_script_name

    def on_modified(self, event):
        if event.is_directory or event.src_path.endswith(self.server_script_name):
            return
        filepath = event.src_path
        print(f"File modified: {filepath}")

    def on_created(self, event):
        if event.is_directory or event.src_path.endswith(self.server_script_name):
            return
        filepath = event.src_path
        print(f"File created: {filepath}")

# Client handler for each HTTP request
def client(connectionSocket, addr):
    try:
        message = connectionSocket.recv(1024).decode()
        print(message)

        if not message or len(message.split()) < 2:
            print("Empty or malformed HTTP request received.")
            connectionSocket.close()
            return

        filename = message.split()[1]
        filepath = f"{directory}/{filename}"

        # Check if the file exists and get the updated content
        if os.path.exists(filepath):
            with open(filepath, "r") as f:
                outputdata = f.read()
            print(f"Serving file: {filepath}")
        else:
            outputdata = "<html><body><h1>404 Not Found</h1></body></html>"
            connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
            connectionSocket.send(outputdata.encode())
            connectionSocket.close()
            return

        # Send HTTP response with the file content
        connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())
        connectionSocket.send(outputdata.encode())
        connectionSocket.send("\r\n".encode())

    except IOError:
        # Send response message for file not found
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
        connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>\r\n".encode())
    finally:
        connectionSocket.close()
        print('Connection closed')

# Start the web server
def start_server(directory, serverPort=12000):
    serverSocket = socket(AF_INET, SOCK_STREAM)
    serverSocket.bind(("", serverPort))
    serverSocket.listen(5)  # Listen for multiple connections
    print(f"Multi-threaded server is running on port {serverPort}...")

    # Watchdog Observer to monitor file changes
    observer = Observer()
    server_script_name = "HTTP_Server.ipynb"  # Prevent reloading the server script itself
    event_handler = FileChangeHandler(directory, server_script_name)
    observer.schedule(event_handler, directory, recursive=True)
    observer.start()

    try:
        while True:
            print('Ready to serve...')
            connectionSocket, addr = serverSocket.accept()
            client_thread = Thread(target=client, args=(connectionSocket, addr))
            client_thread.start()
    except KeyboardInterrupt:
        print("\nServer shutting down...")
    finally:
        observer.stop()
        observer.join()
        serverSocket.close()
        sys.exit()

if __name__ == "__main__":
    directory = "c:/Users/dell/Desktop/github/EEA-Networking101_s/Week 3 - Web Server"
    start_server(directory)

Your dynamic http server is now ready